<center><font size="10"> 🔥NLP🔥 </font></center>

## Char-RNN

#### Generating an Shakespearean Text using Charecter RNN

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline

#### Downloading Shakespeare dataset

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)

In [3]:
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
print(shakespeare_text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

#### Encode the dataset and map them into all different charecter ids

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [7]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [8]:

tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [9]:
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count

In [10]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
encoded

array([19,  5,  8, ..., 20, 26, 10])

#### Splitting the dataset 

In [12]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [13]:

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [17]:
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [19]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [74]:
model = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
        ])
tf.config.run_functions_eagerly(True)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=2)

Epoch 1/2
31368/31368 [==============================] - 2708s 86ms/step - loss: 1.6186
Epoch 2/2
31368/31368 [==============================] - 2702s 86ms/step - loss: 1.5391


In [78]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [82]:
X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'u'

In [14]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [15]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [16]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])


In [17]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [19]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 6s 13ms/step - loss: 1.7120
Epoch 2/50
313/313 [==============================] - 4s 13ms/step - loss: 1.7119
Epoch 3/50
313/313 [==============================] - 4s 13ms/step - loss: 1.7072
Epoch 4/50
313/313 [==============================] - 4s 13ms/step - loss: 1.7069
Epoch 5/50
313/313 [==============================] - 4s 13ms/step - loss: 1.7074
Epoch 6/50
313/313 [==============================] - 4s 13ms/step - loss: 1.7053
Epoch 7/50
313/313 [==============================] - 4s 14ms/step - loss: 1.7036
Epoch 8/50
313/313 [==============================] - 4s 14ms/step - loss: 1.7051
Epoch 9/50
313/313 [==============================] - 5s 15ms/step - loss: 1.7029
Epoch 10/50
313/313 [==============================] - 4s 13ms/step - loss: 1.7052
Epoch 11/50
313/313 [==============================] - 4s 12ms/step - loss: 1.7013
Epoch 12/50
313/313 [==============================] - 4s 13ms/step - loss: 1.7004
Epoch 13/50
3

In [20]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=1000, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [21]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

In [22]:
stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model
tf.random.set_seed(42)

In [24]:
print(complete_text("juliet"))

juliet:
'for it in sweet spirit on my form,,
or this frince of such a age, once hath last talk the dihine
take you blows to beer my clorance.
bear thee, so fine end to be painted.

sampson:
he wile let her brother's would to the duke?
that met see the ring, he chadge, we did stand in that peaced:
indeed-hip in any palour of
mother is the dids, who doats, is't be
out so woend all then the upwerily. i know not s;' come to them die? reason
percy my good,
dreasure we stoik?

juliet:
true, a cloody roparit.
had sack'd? weat minsful thoughts friie, there cunser'd
agom, i was vire yet he won that wilt be
the fouch of full wath beed unkindly.

gremio:
thy greater have been his aster, i am ruins?

gloucester:
but now you'll weep be those that his now down
that do you can a lose forth he many king will be resign,
thennor, but i am a montua
yet eve to the night, it assone many by
sift his omfemant with all soleratien,
he'll have poison of the king, daughter; and never be?
but you
nou
the meaver o